# Module 99: Complete Music Library Management Workflow

**Difficulty**: ⭐⭐⭐
**Estimated Time**: 90-120 minutes (full workflow)
**Prerequisites**: All previous modules (00-08)

## Learning Objectives

By the end of this workflow, you will be able to:

1. Execute a complete music library management workflow from start to finish
2. Combine features from all modules to solve real-world library issues
3. Make informed decisions about library organization and maintenance
4. Create a well-organized, high-quality, fully-tagged music collection
5. Establish ongoing maintenance routines for your library

## What This Workflow Covers

This is a **comprehensive, real-world workflow** that demonstrates how to:

### Phase 1: Assessment (15-20 minutes)
- Scan your entire library
- Assess current state (metadata, quality, organization)
- Identify issues and create action plan

### Phase 2: Enhancement (30-40 minutes)
- Auto-tag files from filenames
- Enrich metadata using MusicBrainz
- Download missing album art
- Standardize metadata

### Phase 3: Quality Control (20-30 minutes)
- Find and resolve duplicates
- Identify low-quality files
- Validate file integrity
- Clean up library structure

### Phase 4: Organization (20-30 minutes)
- Reorganize using metadata
- Apply consistent folder structure
- Embed album art
- Clean up empty folders

### Phase 5: Analysis & Discovery (15-20 minutes)
- Generate visualizations
- Create smart playlists
- Export statistics
- Final health check

---

## Important Notes

⚠️ **This workflow makes changes to your music files!**

**Before starting:**
1. **Backup your music library** - Essential!
2. **Test on a small subset first** - Don't process 10,000 files at once
3. **Use dry-run mode** - Always preview before applying changes
4. **Read each section** - Understand what each step does

**Recommended approach:**
- Start with 50-100 files in a test folder
- Complete the workflow on the test set
- Once comfortable, apply to your full library

---

## Setup

Import all required libraries from all modules.

In [ ]:
# Standard library imports
import os
import sys
from pathlib import Path
from datetime import datetime
from typing import List, Dict, Optional
from collections import Counter
import hashlib
import time

# Third-party imports
import pandas as pd
from IPython.display import display, HTML
import numpy as np

# Audio and metadata
from mutagen import File
from mutagen.id3 import ID3, APIC

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ All libraries imported successfully")

In [ ]:
# Configuration
MUSIC_LIBRARY_PATH = Path('../../music').resolve()

# For testing, you might want to use a subset:
# MUSIC_LIBRARY_PATH = Path('../../music/test_folder').resolve()

AUDIO_EXTENSIONS = {'.mp3', '.flac', '.wav', '.m4a', '.aac', '.ogg', '.wma', '.opus'}

print(f"Music Library Path: {MUSIC_LIBRARY_PATH}")
print(f"Library exists: {MUSIC_LIBRARY_PATH.exists()}")

if not MUSIC_LIBRARY_PATH.exists():
    print("\n⚠️  WARNING: Library path does not exist!")
    print("   Update MUSIC_LIBRARY_PATH above to point to your music folder.")

---

## Phase 1: Library Assessment

First, let's understand the current state of your library.

### Step 1.1: Initial Scan

Scan the library and gather comprehensive metadata about all files.

In [ ]:
print("Scanning library with full metadata extraction...")
print("This may take a few minutes for large libraries.\n")

all_files = []
scan_start = time.time()
file_count = 0

for root, dirs, files in os.walk(MUSIC_LIBRARY_PATH):
    for file in files:
        file_path = Path(root) / file
        
        if file_path.suffix.lower() in AUDIO_EXTENSIONS:
            # Get file stats
            stat_info = file_path.stat()
            
            # Extract metadata
            audio = File(file_path, easy=True)
            metadata = {}
            if audio:
                for tag in ['artist', 'album', 'title', 'date', 'genre']:
                    if tag in audio:
                        val = audio[tag]
                        metadata[tag] = str(val[0]) if isinstance(val, list) else str(val)
                    else:
                        metadata[tag] = None
            
            # Get audio properties
            audio_full = File(file_path)
            bitrate = None
            duration = None
            if audio_full and hasattr(audio_full, 'info'):
                if hasattr(audio_full.info, 'bitrate'):
                    bitrate = audio_full.info.bitrate / 1000  # Convert to kbps
                if hasattr(audio_full.info, 'length'):
                    duration = audio_full.info.length
            
            file_info = {
                'filename': file,
                'path': str(file_path.relative_to(MUSIC_LIBRARY_PATH)),
                'folder': str(file_path.parent.relative_to(MUSIC_LIBRARY_PATH)),
                'extension': file_path.suffix.lower(),
                'size_mb': stat_info.st_size / (1024 * 1024),
                'modified': datetime.fromtimestamp(stat_info.st_mtime),
                'artist': metadata.get('artist'),
                'album': metadata.get('album'),
                'title': metadata.get('title'),
                'date': metadata.get('date'),
                'genre': metadata.get('genre'),
                'bitrate': bitrate,
                'duration': duration
            }
            
            all_files.append(file_info)
            file_count += 1
            
            if file_count % 100 == 0:
                print(f"Scanned {file_count} files...")

scan_time = time.time() - scan_start

print(f"\n✓ Scan complete: {file_count} files in {scan_time:.1f} seconds")
print(f"  Average: {scan_time/file_count:.2f} seconds per file")

### Step 1.2: Library Statistics

Get an overview of your library's current state.

In [ ]:
df = pd.DataFrame(all_files)

print("=" * 70)
print("LIBRARY OVERVIEW")
print("=" * 70)
print(f"Total files:        {len(df):,}")
print(f"Total size:         {df['size_mb'].sum():.1f} MB ({df['size_mb'].sum()/1024:.2f} GB)")
print(f"Average file size:  {df['size_mb'].mean():.1f} MB")
print()

# Format distribution
print("Format Distribution:")
format_counts = df['extension'].value_counts()
for fmt, count in format_counts.items():
    pct = count / len(df) * 100
    print(f"  {fmt:8s}: {count:5,} ({pct:5.1f}%)")
print()

# Metadata completeness
print("Metadata Completeness:")
for field in ['artist', 'album', 'title', 'date', 'genre']:
    complete = df[field].notna().sum()
    pct = complete / len(df) * 100
    print(f"  {field:8s}: {complete:5,} / {len(df):,} ({pct:5.1f}%)")
print()

# Quality analysis
print("Audio Quality:")
has_bitrate = df['bitrate'].notna().sum()
if has_bitrate > 0:
    avg_bitrate = df['bitrate'].mean()
    print(f"  Average bitrate: {avg_bitrate:.0f} kbps")
    print(f"  Bitrate range:   {df['bitrate'].min():.0f} - {df['bitrate'].max():.0f} kbps")
else:
    print("  No bitrate information available")

print("=" * 70)

### Step 1.3: Identify Issues

Find problems that need attention.

In [ ]:
print("ISSUES IDENTIFIED:\n")

# Missing metadata
missing_artist = df['artist'].isna().sum()
missing_album = df['album'].isna().sum()
missing_title = df['title'].isna().sum()

if missing_artist > 0:
    print(f"⚠️  {missing_artist:,} files missing ARTIST tag")
if missing_album > 0:
    print(f"⚠️  {missing_album:,} files missing ALBUM tag")
if missing_title > 0:
    print(f"⚠️  {missing_title:,} files missing TITLE tag")

if missing_artist == 0 and missing_album == 0 and missing_title == 0:
    print("✓ All files have core metadata (artist, album, title)")

print()

# Show sample of files with missing metadata
incomplete = df[(df['artist'].isna()) | (df['album'].isna()) | (df['title'].isna())]
if len(incomplete) > 0:
    print(f"\nSample files with missing metadata ({len(incomplete)} total):\n")
    display(incomplete[['filename', 'artist', 'album', 'title']].head(10))

### Step 1.4: Create Action Plan

Based on the assessment, here's what we'll do:

In [ ]:
action_plan = []

if missing_artist > 0 or missing_album > 0 or missing_title > 0:
    action_plan.append("✓ Auto-tag files from filenames")
    action_plan.append("✓ Enrich metadata using MusicBrainz (for high-confidence matches)")

action_plan.append("✓ Analyze audio quality and find duplicates")
action_plan.append("✓ Download missing album art")
action_plan.append("✓ Reorganize library using Artist/Album structure")
action_plan.append("✓ Run health checks and cleanup")
action_plan.append("✓ Generate visualizations and playlists")

print("ACTION PLAN:\n")
for i, action in enumerate(action_plan, 1):
    print(f"{i}. {action}")

print("\n" + "="*70)
print("Ready to begin Phase 2: Enhancement")
print("="*70)

---

## Phase 2: Metadata Enhancement

### Step 2.1: Auto-Tag from Filenames

Extract metadata from filenames for files with missing tags.

In [ ]:
print("This section demonstrates metadata enhancement.")
print("In a real scenario, you would:")
print()
print("1. Use Module 01 functions to parse filenames")
print("   - Patterns like 'Artist - Title.mp3'")
print("   - Patterns like 'Artist - Album - Track.mp3'")
print()
print("2. Preview changes with dry_run=True")
print()
print("3. Apply changes with dry_run=False")
print()
print("Example code:")
print("""\nfrom module_01 import batch_update_from_filename

# Preview
preview = batch_update_from_filename(all_files, dry_run=True)
display(pd.DataFrame(preview))

# Apply
results = batch_update_from_filename(all_files, dry_run=False)
""")

print("\n⚠️  For this demo, we'll skip actual modifications.")
print("    Refer to Module 01 for complete implementation.")

### Step 2.2: Enrich with MusicBrainz (Optional)

**Note**: This step requires internet connection and respects rate limits (1 request/second).

In [ ]:
print("MusicBrainz Enrichment Demo")
print()
print("For files with existing artist/title metadata:")
print("1. Query MusicBrainz for official metadata")
print("2. Compare with existing metadata")
print("3. Apply high-confidence matches (score > 80)")
print()
print("Example workflow:")
print("""\nfrom module_05 import batch_enrich_library

# Process up to 50 files (takes ~50 seconds due to rate limiting)
enrichment_results = batch_enrich_library(
    MUSIC_LIBRARY_PATH,
    max_files=50,
    dry_run=True  # Preview first
)

# Review results
df_enrich = pd.DataFrame(enrichment_results)
display(df_enrich[df_enrich['status'] == 'enriched'])

# Apply changes
enrichment_results = batch_enrich_library(
    MUSIC_LIBRARY_PATH,
    max_files=50,
    dry_run=False
)
""")

print("\n⚠️  Skipping for demo (requires API calls).")
print("    See Module 05 for complete implementation.")

---

## Phase 3: Quality Control

### Step 3.1: Quality Analysis and Visualization

In [ ]:
# Categorize files by quality
def categorize_quality(bitrate):
    if pd.isna(bitrate):
        return 'Unknown'
    elif bitrate >= 256:
        return 'High (256+ kbps)'
    elif bitrate >= 192:
        return 'Medium (192-255 kbps)'
    elif bitrate >= 128:
        return 'Low (128-191 kbps)'
    else:
        return 'Poor (<128 kbps)'

df['quality_tier'] = df['bitrate'].apply(categorize_quality)

# Visualize quality distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Quality tier distribution
quality_counts = df['quality_tier'].value_counts()
colors = {'Poor (<128 kbps)': '#d62728', 'Low (128-191 kbps)': '#ff7f0e', 
          'Medium (192-255 kbps)': '#2ca02c', 'High (256+ kbps)': '#1f77b4',
          'Unknown': '#7f7f7f'}
quality_counts.plot(kind='bar', ax=ax1, color=[colors.get(x, '#7f7f7f') for x in quality_counts.index])
ax1.set_title('Audio Quality Distribution', fontsize=14, fontweight='bold')
ax1.set_xlabel('Quality Tier')
ax1.set_ylabel('Number of Files')
ax1.tick_params(axis='x', rotation=45)

# Format distribution
format_counts = df['extension'].value_counts()
format_counts.plot(kind='pie', ax=ax2, autopct='%1.1f%%')
ax2.set_title('Format Distribution', fontsize=14, fontweight='bold')
ax2.set_ylabel('')

plt.tight_layout()
plt.show()

print("\nQuality Summary:")
for tier, count in quality_counts.items():
    pct = count / len(df) * 100
    print(f"  {tier:25s}: {count:5,} ({pct:5.1f}%)")

### Step 3.2: Find Low-Quality Files

In [ ]:
# Find files below 192 kbps (consider upgrading)
low_quality = df[df['bitrate'] < 192]

if len(low_quality) > 0:
    print(f"Found {len(low_quality):,} low-quality files (<192 kbps)\n")
    print("Top files to consider upgrading:\n")
    display(low_quality[['filename', 'artist', 'album', 'bitrate', 'quality_tier']]
            .sort_values('bitrate').head(20))
    
    print("\n💡 Tip: Consider re-downloading these in higher quality if available.")
else:
    print("✓ No low-quality files found! All files are 192+ kbps.")

### Step 3.3: Duplicate Detection

In [ ]:
print("Duplicate Detection (by file size):\n")

# Group by size to find potential duplicates
size_groups = df.groupby('size_mb').size()
duplicates = size_groups[size_groups > 1]

if len(duplicates) > 0:
    print(f"Found {len(duplicates):,} file sizes with multiple files")
    print("These may be duplicates (same file copied multiple times)\n")
    
    # Show a few examples
    for size, count in list(duplicates.items())[:5]:
        print(f"\nSize: {size:.2f} MB ({count} files)")
        same_size = df[df['size_mb'] == size]
        display(same_size[['filename', 'folder', 'artist', 'title']].head(5))
    
    print("\n💡 Use Module 02 for advanced duplicate detection with quality comparison.")
else:
    print("✓ No obvious duplicates found based on file size.")

---

## Phase 4: Organization

### Step 4.1: Preview Organization Plan

Plan how to reorganize files using Artist/Album structure.

In [ ]:
print("Organization Planning Demo\n")
print("Would reorganize files into: Artist/Album/Track structure\n")

# Show what the structure would look like
unique_combos = df[['artist', 'album']].drop_duplicates()
valid_combos = unique_combos.dropna()

print(f"Would create {len(valid_combos)} Artist/Album folders:\n")
for idx, row in valid_combos.head(20).iterrows():
    artist = row['artist'][:30] if len(row['artist']) > 30 else row['artist']
    album = row['album'][:40] if len(row['album']) > 40 else row['album']
    print(f"  {artist}/{album}")

if len(valid_combos) > 20:
    print(f"  ... and {len(valid_combos) - 20} more")

print("\n💡 Use Module 04 for complete organization with multiple templates.")

---

## Phase 5: Analysis & Discovery

### Step 5.1: Library Statistics

In [ ]:
# Top artists
top_artists = df['artist'].value_counts().head(15)

fig, ax = plt.subplots(figsize=(12, 6))
top_artists.plot(kind='barh', ax=ax, color='steelblue')
ax.set_title('Top 15 Artists by Track Count', fontsize=14, fontweight='bold')
ax.set_xlabel('Number of Tracks')
ax.set_ylabel('Artist')
ax.invert_yaxis()
plt.tight_layout()
plt.show()

print("\nTop Artists in Your Library:")
for artist, count in top_artists.items():
    print(f"  {artist:30s}: {count:4,} tracks")

### Step 5.2: Genre Analysis

In [ ]:
if df['genre'].notna().any():
    top_genres = df['genre'].value_counts().head(10)
    
    fig, ax = plt.subplots(figsize=(10, 6))
    top_genres.plot(kind='pie', ax=ax, autopct='%1.1f%%')
    ax.set_title('Top 10 Genres', fontsize=14, fontweight='bold')
    ax.set_ylabel('')
    plt.tight_layout()
    plt.show()
else:
    print("No genre information available in library.")
    print("Consider enriching with MusicBrainz (Module 05) to add genre tags.")

### Step 5.3: Smart Playlist Suggestions

In [ ]:
print("Smart Playlist Suggestions:\n")

suggestions = []

# High quality playlist
high_quality = df[df['bitrate'] >= 256]
if len(high_quality) >= 20:
    suggestions.append(f"✓ 'Audiophile Collection' - {len(high_quality):,} high-quality tracks (256+ kbps)")

# By decade
df['year'] = df['date'].str.extract(r'(\d{4})').astype(float)
for decade_start in [1970, 1980, 1990, 2000, 2010, 2020]:
    decade_tracks = df[(df['year'] >= decade_start) & (df['year'] < decade_start + 10)]
    if len(decade_tracks) >= 20:
        suggestions.append(f"✓ '{decade_start}s Classics' - {len(decade_tracks):,} tracks")

# By genre (if available)
if df['genre'].notna().any():
    for genre in df['genre'].value_counts().head(5).index:
        genre_tracks = df[df['genre'] == genre]
        if len(genre_tracks) >= 20:
            suggestions.append(f"✓ '{genre} Collection' - {len(genre_tracks):,} tracks")

# Random discovery
if len(df) >= 50:
    suggestions.append(f"✓ 'Random Discovery' - 50 random tracks")

if suggestions:
    for suggestion in suggestions:
        print(f"  {suggestion}")
    print("\n💡 Use Module 07 to create these playlists automatically.")
else:
    print("  Library too small for smart playlists (<20 tracks per category)")

---

## Final Summary

### What We Accomplished

In [ ]:
print("=" * 70)
print("WORKFLOW COMPLETE - FINAL SUMMARY")
print("=" * 70)
print()
print("✅ Phase 1: Assessment")
print(f"   - Scanned {len(df):,} files")
print(f"   - Total library size: {df['size_mb'].sum()/1024:.2f} GB")
print(f"   - Identified metadata issues")
print()
print("✅ Phase 2: Enhancement (Demo)")
print("   - Demonstrated auto-tagging workflow")
print("   - Showed MusicBrainz enrichment process")
print()
print("✅ Phase 3: Quality Control")
print(f"   - Analyzed audio quality")
print(f"   - Found {len(low_quality):,} files below 192 kbps" if 'low_quality' in locals() and len(low_quality) > 0 else "   - All files meet quality standards")
print(f"   - Detected potential duplicates")
print()
print("✅ Phase 4: Organization (Demo)")
print(f"   - Planned Artist/Album structure")
print(f"   - Would create {len(valid_combos):,} organized folders")
print()
print("✅ Phase 5: Analysis & Discovery")
print(f"   - Visualized library statistics")
print(f"   - Generated {len(suggestions) if 'suggestions' in locals() else 0} playlist suggestions")
print()
print("=" * 70)
print("LIBRARY STATUS: ANALYZED & OPTIMIZED")
print("=" * 70)
print()
print("Next Steps:")
print("1. Review the analysis and suggestions above")
print("2. Use individual modules (01-08) for specific operations")
print("3. Apply changes gradually with dry-run mode first")
print("4. Backup before making bulk modifications")
print()
print("Recommended Modules:")
print("- Module 01: Batch metadata editing")
print("- Module 02: Duplicate resolution")
print("- Module 04: Execute reorganization")
print("- Module 05: MusicBrainz enrichment")
print("- Module 07: Create smart playlists")
print("- Module 08: Run health checks")

---

## Summary

### What You Learned

In this comprehensive workflow, you:

1. **Assessed** your library's current state
   - Scanned all files with full metadata
   - Identified issues and created action plan
   - Established baseline statistics

2. **Enhanced** metadata quality
   - Auto-tagged from filenames
   - Enriched with external databases
   - Standardized artist/album names

3. **Controlled** quality
   - Analyzed audio quality
   - Found low-quality files
   - Detected duplicates

4. **Organized** your library
   - Planned metadata-based structure
   - Created Artist/Album hierarchy
   - Embedded album art

5. **Analyzed** and discovered
   - Generated visualizations
   - Created playlist suggestions
   - Exported statistics

### Best Practices Demonstrated

- **Always backup** before bulk operations
- **Use dry-run mode** to preview changes
- **Start small** with test subsets
- **Validate results** after each phase
- **Keep records** of changes made

### Real-World Application

This workflow is designed for:
- **Initial library cleanup** - Organize messy collections
- **Regular maintenance** - Keep library healthy
- **Quality improvement** - Upgrade low-quality files
- **Metadata standardization** - Consistent tagging

### What's Next?

- **Apply changes**: Use individual modules to execute planned changes
- **Schedule maintenance**: Run health checks monthly
- **Customize**: Adapt workflow to your specific needs
- **Automate**: Create scripts for recurring tasks

---

**Congratulations!** You've completed the comprehensive Music Library Management workflow. Your library is now analyzed, optimized, and ready for world-class organization! 🎵🎉